In [1]:
import pandas as pd
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.preprocessors import data_split, FeatureEngineer,YahooDownloader
from finrl.plot import backtest_stats, get_baseline , plot_return

C:\Users\Win 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2024-01-01'
PATH = '/files'
SYMBOLS = ['msft','ibm','hd','cat','amzn','intc','t','v','gs']
# INDICATORS = TECHNICAL_INDICATORS_LIST
RESULTS_DIR = '/results'
TIMESTEPS = 150000
date_col = "date"
tic_col = "tic"

In [3]:
final_data = pd.read_csv("Processed_data/finaldata.csv")

In [4]:
trade_data = data_split(final_data, TRADE_START_DATE,TRADE_END_DATE)

In [5]:
trade_data

,Unnamed: 0,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,34479,2020-07-01,amzn,137.899506,144.750000,137.699997,143.934998,127268000.0,2.0,4.503504,142.937010,121.837240,67.910978,160.233497,42.384894,129.056634,122.777067,28.620001,15.796694
0,34480,2020-07-01,cat,129.380005,129.399994,125.879997,116.095177,2807800.0,2.0,1.224568,125.691200,109.072293,52.865421,35.797066,11.101666,114.903159,109.354758,28.620001,15.796694
0,34481,2020-07-01,gs,198.880005,199.740005,195.520004,179.992264,2620100.0,2.0,0.564420,202.785763,170.719414,50.962750,-35.698750,4.814324,183.791767,172.839070,28.620001,15.796694
0,34482,2020-07-01,hd,249.649994,250.330002,246.929993,226.875961,3677300.0,2.0,2.174400,236.307821,217.950526,55.509709,20.619700,11.195207,225.850883,211.615886,28.620001,15.796694
0,34483,2020-07-01,ibm,114.980881,115.898659,113.164436,94.427551,4869967.0,2.0,-1.008463,107.123203,89.484289,47.489713,-74.240217,5.067577,98.343576,96.945911,28.620001,15.796694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,45958,2023-12-28,ibm,163.960007,163.960007,163.399994,162.270599,2071300.0,3.0,2.599378,163.590840,157.356129,67.480249,84.780613,35.160185,158.008128,149.200041,12.470000,1.497744
879,45959,2023-12-28,intc,50.810001,50.869999,50.160000,50.242729,27705200.0,3.0,2.269009,50.961699,39.588882,68.839558,200.604792,50.602738,44.708619,40.466726,12.470000,1.497744
879,45960,2023-12-28,msft,375.369995,376.459991,374.160004,374.587280,14327000.0,3.0,2.668700,377.711291,366.355744,58.167714,44.673886,23.670772,373.178901,355.870526,12.470000,1.497744
879,45961,2023-12-28,t,16.540001,16.870001,16.530001,16.205858,32112900.0,3.0,0.140437,16.564676,15.693114,58.594741,62.531131,22.595007,15.944796,15.294035,12.470000,1.497744


In [5]:
stock_dimension = len(trade_data.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
buy_cost_list = sell_cost_list = [0.001]*stock_dimension
num_stock_shares = [0] * stock_dimension

Stock Dimension: 9, State Space: 91


In [6]:
initial_amount = 1000000
env_kwargs = {
    "hmax": 100,
    "initial_amount": initial_amount,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    # "initial_total_asset": initial_amount
    }

In [7]:
e_trade_gym = StockTradingEnv(
        df= trade_data,
        turbulence_threshold=70,
        risk_indicator_col="vix",
        **env_kwargs,
    )

In [8]:
from stable_baselines3 import A2C,DDPG,SAC,TD3,PPO

In [9]:
if_store_actions = True
if_store_result = True
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_sac = True
if_using_td3 = True

In [10]:
trained_a2c = A2C.load("save_models\\a2c.zip")
trained_ddpg= DDPG.load("save_models\\ddpg.zip")
trained_td3= TD3.load("save_models\\td3.zip")
trained_sac= SAC.load("save_models\\sac.zip")
trained_ppo= PPO.load("save_models\\ppo.zip")

In [11]:
if if_using_a2c:
        result_a2c, actions_a2c = DRLAgent.DRL_prediction(
            model=trained_a2c, environment=e_trade_gym
        )


if if_using_ddpg:
        result_ddpg, actions_ddpg = DRLAgent.DRL_prediction(
            model=trained_ddpg, environment=e_trade_gym
        )


if if_using_ppo:
        result_ppo, actions_ppo = DRLAgent.DRL_prediction(
            model=trained_ppo, environment=e_trade_gym
            
        )


if if_using_sac:
        result_sac, actions_sac = DRLAgent.DRL_prediction(
            model=trained_sac, environment=e_trade_gym
        )


if if_using_td3:
        result_td3, actions_td3 = DRLAgent.DRL_prediction(
            model=trained_td3, environment=e_trade_gym
        )


hit end!
hit end!
hit end!
hit end!
hit end!


In [12]:
if if_store_actions:
    actions_a2c.to_excel("action_records\\actions_a2c.xlxs") if if_using_a2c else None
    actions_ddpg.to_excel("action_records\\actions_ddpg.xlsx") if if_using_ddpg else None
    actions_td3.to_excel("action_records\\actions_td3.xlxs") if if_using_td3 else None
    actions_ppo.to_excel("action_records\\actions_ppo.xlxs") if if_using_ppo else None
    actions_sac.to_excel("action_records\\actions_sac.xlxs") if if_using_sac else None

In [13]:
def get_comparison_tic(ticker,start_date,end_date):
        data_ = get_baseline(ticker=ticker,start = start_date,end=end_date)
        data = pd.DataFrame()
        data[date_col]=data_[date_col]
        data[ticker] = data_["close"]
        '''# select the rows between trade_start and trade_end (not included),
        since some values may not in this region'''
        data = data.loc[(data[date_col]>= start_date) & (data[date_col]<end_date)]
        return data

In [14]:
print(set(final_data["tic"]))

{'msft', 'v', 'ibm', 'cat', 'amzn', 't', 'gs', 'intc', 'hd'}


In [25]:
comparison_tic_name = "amzn"
result = get_comparison_tic(ticker=comparison_tic_name,start_date=TRADE_START_DATE,end_date=TRADE_END_DATE)

[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (881, 8)


In [26]:
if if_using_a2c:
        result_a2c.rename(columns={"account_value": "A2C"}, inplace=True)
        result = pd.merge(result, result_a2c, how="left")
if if_using_ddpg:
        result_ddpg.rename(columns={"account_value": "DDPG"}, inplace=True)
        result = pd.merge(result, result_ddpg, how="left")
if if_using_td3:
        result_td3.rename(columns={"account_value": "TD3"}, inplace=True)
        result = pd.merge(result, result_td3, how="left")
if if_using_ppo:
        result_ppo.rename(columns={"account_value": "PPO"}, inplace=True)
        result = pd.merge(result, result_ppo, how="left")
if if_using_sac:
        result_sac.rename(columns={"account_value": "SAC"}, inplace=True)
        result = pd.merge(result, result_sac, how="left")

    # remove the rows with nan
result = result.dropna(axis=0, how="any")

In [27]:

    # calc the column name of strategies, including amzn
col_strategies = []
for col in result.columns:
        if col != date_col and col != "" and "Unnamed" not in col:
            col_strategies.append(col)

    # make sure that the first row of DJI is initial_amount
col = comparison_tic_name
result[col] = result[col] / result[col].iloc[0] * initial_amount
result = result.reset_index(drop=True)
if if_store_result:
        result.to_excel("H:\TradingBot\\result.xlsx")

In [28]:
from finrl.plot import backtest_stats, get_baseline , plot_return,plot_result
result = pd.read_excel("results\\result.xlsx")
# calc the column name of strategies, including DJI
col_strategies = []
for col in result.columns:
    if col != date_col and col != "" and "Unnamed" not in col:
        col_strategies.append(col)
for col in col_strategies:
    stats = backtest_stats(result, value_col_name=col)
    print("\nstats of " + col + ": \n", stats)
    print("result: ", result)
if if_store_result:
        result.to_excel("results\\result.xlsx")

# plot fig
plot_result(
        result=result,
        column_as_x=date_col,
        # if_need_calc_return=True,
        savefig_filename="results\\stock_trading.png",
        xlabel="Date",
        ylabel="Return",
        # if_transfer_date=True,
        num_days_xticks=20,
    )

Annual return          0.018367
Cumulative returns     0.065620
Annual volatility      0.372707
Sharpe ratio           0.234926
Calmar ratio           0.032713
Stability              0.389070
Max drawdown          -0.561453
Omega ratio            1.041934
Sortino ratio          0.339579
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.083052
Daily value at risk   -0.046609
dtype: float64

stats of amzn: 
 Annual return          0.018367
Cumulative returns     0.065620
Annual volatility      0.372707
Sharpe ratio           0.234926
Calmar ratio           0.032713
Stability              0.389070
Max drawdown          -0.561453
Omega ratio            1.041934
Sortino ratio          0.339579
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.083052
Daily value at risk   -0.046609
dtype: float64
result:       Unnamed: 0        date          amzn           A2C          DDPG  \
0             0  2020-07-01  1.000000e

C:\Users\Win 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\finrl\plot.py:240: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
